In [9]:
import numpy as np
import random
from sklearn.datasets.samples_generator import make_regression 
import pylab
from scipy import stats
import pandas as pd

In [10]:
data = pd.read_csv( 'adult.csv', delimiter = ',' )
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [11]:
#так как здесь вместо nan просто '?' то прийдется поменять его на nan и затем дропнуть nan

In [12]:
data = data.replace('?', np.nan)
data = data.dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 48841
Data columns (total 15 columns):
age                45222 non-null int64
workclass          45222 non-null object
fnlwgt             45222 non-null int64
education          45222 non-null object
educational-num    45222 non-null int64
marital-status     45222 non-null object
occupation         45222 non-null object
relationship       45222 non-null object
race               45222 non-null object
gender             45222 non-null object
capital-gain       45222 non-null int64
capital-loss       45222 non-null int64
hours-per-week     45222 non-null int64
native-country     45222 non-null object
income             45222 non-null object
dtypes: int64(6), object(9)
memory usage: 5.5+ MB


In [13]:
#выбираю фичи по которым буду строить модель

In [14]:
data.drop(["capital-loss", "capital-gain", "educational-num","fnlwgt"],axis = 1, inplace = True)
data.head()

,age,workclass,education,marital-status,occupation,relationship,race,gender,hours-per-week,native-country,income
0,25,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,40,United-States,<=50K
1,38,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,40,United-States,>50K
3,44,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,40,United-States,>50K
5,34,Private,10th,Never-married,Other-service,Not-in-family,White,Male,30,United-States,<=50K


In [15]:
selectedColumns = data[ [ 'age', 'workclass', 'education', "occupation","marital-status", "relationship" ,"race","gender","hours-per-week", "native-country", "income" ] ]


In [16]:
selectedColumns.head()

,age,workclass,education,occupation,marital-status,relationship,race,gender,hours-per-week,native-country,income
0,25,Private,11th,Machine-op-inspct,Never-married,Own-child,Black,Male,40,United-States,<=50K
1,38,Private,HS-grad,Farming-fishing,Married-civ-spouse,Husband,White,Male,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,Protective-serv,Married-civ-spouse,Husband,White,Male,40,United-States,>50K
3,44,Private,Some-college,Machine-op-inspct,Married-civ-spouse,Husband,Black,Male,40,United-States,>50K
5,34,Private,10th,Other-service,Never-married,Not-in-family,White,Male,30,United-States,<=50K


In [17]:
#отделяю Y и удаляю его из датасета

In [18]:
Y = selectedColumns[["income"]]
Y.head()
del selectedColumns["income"]

In [19]:
selectedColumns.head()

,age,workclass,education,occupation,marital-status,relationship,race,gender,hours-per-week,native-country
0,25,Private,11th,Machine-op-inspct,Never-married,Own-child,Black,Male,40,United-States
1,38,Private,HS-grad,Farming-fishing,Married-civ-spouse,Husband,White,Male,50,United-States
2,28,Local-gov,Assoc-acdm,Protective-serv,Married-civ-spouse,Husband,White,Male,40,United-States
3,44,Private,Some-college,Machine-op-inspct,Married-civ-spouse,Husband,Black,Male,40,United-States
5,34,Private,10th,Other-service,Never-married,Not-in-family,White,Male,30,United-States


In [20]:
#так как у нас присутствуют категориальные переменные - делаем one hot encoding

In [21]:
X = pd.get_dummies( selectedColumns, columns = ['workclass', "education", "occupation","race","marital-status","relationship", "gender", "native-country"   ] )




In [22]:
#так как в Y у нас по сути текс - закодим его в 1 и 0

In [23]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit( Y["income"] )
le.classes_
le.transform( [ '<=50K', '>50K' ] )

Y = pd.Series( data = le.transform( Y['income'] ) )
Y.head()

0    0
1    0
2    1
3    1
4    0
dtype: int64

In [24]:
# разделяем выборку на треин и тест

In [25]:
from sklearn.model_selection import train_test_split


In [26]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2)

In [27]:
# теперь нужно нормализовать данные

In [28]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [29]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [30]:
from sklearn.linear_model import LogisticRegression

In [31]:
model = LogisticRegression() 

In [32]:
# обучаем модель на треин данных

In [33]:
model.fit( X_train, y_train )
model.score(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8332642286535644

In [34]:
predictions = model.predict_proba( X_test )

In [35]:
model.score(X_test, y_test)

0.8285240464344942

In [52]:
#Посмотрим ради интереса на важность фич

In [51]:
for col, val in zip(pd.DataFrame(X).columns, model.coef_[0]):
    print("{:30} {:.2f}".format(col, val))



age                            0.38
hours-per-week                 0.37
workclass_Federal-gov          0.10
workclass_Local-gov            -0.02
workclass_Private              0.06
workclass_Self-emp-inc         0.05
workclass_Self-emp-not-inc     -0.12
workclass_State-gov            -0.06
workclass_Without-pay          -0.02
education_10th                 -0.20
education_11th                 -0.21
education_12th                 -0.08
education_1st-4th              -0.13
education_5th-6th              -0.14
education_7th-8th              -0.23
education_9th                  -0.17
education_Assoc-acdm           0.04
education_Assoc-voc            0.03
education_Bachelors            0.30
education_Doctorate            0.20
education_HS-grad              -0.17
education_Masters              0.27
education_Preschool            -0.07
education_Prof-school          0.26
education_Some-college         -0.01
occupation_Adm-clerical        -0.01
occupation_Armed-Forces        0.00
occupation_Cr

In [62]:
#занимательно)

In [53]:
# я не очень понял что за перебор параметров необходимо сделать и остановился на том что это нужно сделать обучение с крос валидацией

In [54]:
from sklearn.model_selection import GridSearchCV

In [55]:
cs = 10**np.linspace(-3,1,5)
cs

array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01])

In [56]:
grid = {'C': cs}
gridsearch = GridSearchCV(LogisticRegression(), grid, scoring='accuracy', cv=5)

In [57]:
%%time
gridsearch.fit(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  

CPU times: user 30.7 s, sys: 809 ms, total: 31.6 s
Wall time: 19 s


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [59]:
gridsearch.best_params_

{'C': 0.01}

In [60]:
best_C = gridsearch.best_params_["C"]

In [61]:
#вот мы узнали что лучший парметр C = 0.01  Проверим улучшится ли показатель score с новым параметром

In [78]:
model2 = LogisticRegression(C=best_C) 

In [79]:
model2.fit( X_train, y_train )
model2.score(X_train, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8331536611659341

In [80]:
predictions = model2.predict_proba( X_test )
model2.score(X_test, y_test)

0.8287451630735213

In [68]:
# Почемуто у меня не видно улучшения результатов с перебором параметров по сравнению если брать без него. Я допустил какуюто ошибку?